In [ ]:
# made by Gibeom LEE, HI LAB
# ImageJ와 같은 영상 분석 툴입니다.
# 민감도를 조절한 후, 각 프레임을 사진으로 저장할 수 있습니다.

# space bar: 일시정지/재시작
# s: 모든 프레임 사진 저장
# q: 종료


# 트랙바 설명
#R_MIN : 픽셀의 빨강 채널 값이 이 값 이상일 때만 선택
#G_MIN : 픽셀의 초록 채널 값이 이 값 이상일 때만 선택
#B_MIN : 픽셀의 파랑 채널 값이 이 값 이상일 때만 선택
#BLUR_K : 영상에 중간값 블러(median blur)를 적용해 노이즈를 줄이는 강도 (커널 크기)
#MORPH_K : 마스크에 열림/닫힘(morphology) 연산을 적용해 작은 잡영역 제거 및 구멍 메우기 강도 (커널 크기)
#LargestOnly : 여러 선택 영역 중 가장 큰 영역만 남길지 여부 (0=모두 유지, 1=가장 큰 것만 유지)
#InvertMask : 마스크를 반전해 선택 영역을 뒤집을지 여부 (0=그대로, 1=반전)
#SideCutPercent : 화면 좌우 몇 %를 잘라내고 중앙 영역만 유지할지 조절 (0=전체, 40=양쪽 40% 잘라냄)

In [26]:
import cv2, time, os, json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob  # 폴더 내 파일 리스트 가져오는 라이브러리

# --- 사용자 설정 ---
# 1) 비디오 파일 경로
video_path = Path(r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250901_영상\2try\영상분석\Aloe분석.mp4")

# 2) 영상 가로의 실제 길이(mm) → 픽셀-면적 환산용
side_length_mm = 66.0

# --- 저장 프레임 간격 설정 ---
SAVE_EVERY = 5       # 이 값의 배수 프레임만 이미지 저장 (예: 10이면 10,20,30,...)
SAVE_OFFSET = 0       # 시작 오프셋 (예: 5면 5,15,25,...)
CSV_ONLY_SAVED = False  # True면 CSV에도 저장한 프레임만 기록, False면 모든 프레임 기록
EXCLUDE_FRAME0 = True   # ← (NEW) 0번 프레임 저장 제외
DEBUG_PRINT_SAVED = True  # ← (NEW) 저장되는 프레임 인덱스 매번 출력

In [24]:
base_folder   = video_path.parent / "video_analysis"
# base_folder   = Path("C:/va_out") / video_path.stem  # 경로 이슈 시 권장
Original_folder = base_folder / "Original"
Binary_folder  = base_folder / "Binary"
csv_path      = base_folder / f"{video_path.stem}_area.csv"
params_path   = base_folder / "last_params.json"  # ← 슬라이더 저장 파일

base_folder.mkdir(parents=True, exist_ok=True)

def odd_from_slider(x: int) -> int:
    if x <= 1: return 0
    v = x if x % 2 == 1 else x - 1
    return max(3, v)

def win_longpath(path: Path) -> str:
    s = str(path)
    if os.name == "nt":
        if len(s) >= 2 and s[1] == ":" and not s.startswith("\\\\?\\"):
            return "\\\\?\\" + s
    return s

def safe_write_image(path: Path, img) -> bool:
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"[ERR] make dir failed: {e}")
        return False
    ok = cv2.imwrite(win_longpath(path), img)
    if ok:
        return True
    try:
        ext = (path.suffix or ".png").lower()
        if ext not in [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif", ".webp"]:
            ext = ".png"
        ret, buf = cv2.imencode(ext, img)
        if ret:
            buf.tofile(win_longpath(path))
            return True
        else:
            print(f"[ERR] imencode failed for: {path}")
    except Exception as e:
        print(f"[ERR] fallback write failed for {path}: {e}")
    return False

def preprocess_and_mask(Original_bgr, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, use_largest):
    if BLUR_K >= 3:
        Original_bgr = cv2.medianBlur(Original_bgr, BLUR_K)

    rgb = cv2.cvtColor(Original_bgr, cv2.COLOR_BGR2RGB)
    R = rgb[:, :, 0]
    G = rgb[:, :, 1]
    B = rgb[:, :, 2]

    # 절대 임계값 방식
    cond = (R >= R_MIN) & (G >= G_MIN) & (B >= B_MIN)

    mask = np.zeros(G.shape, dtype=np.uint8)
    mask[cond] = 255

    if MORPH_K >= 3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (MORPH_K, MORPH_K))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=1)

    if use_largest:
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
        if num_labels > 1:
            areas = stats[1:, cv2.CC_STAT_AREA]
            largest = 1 + np.argmax(areas)
            mask = np.where(labels == largest, 255, 0).astype(np.uint8)
        else:
            mask[:] = 0
    return mask



def overlay_mask(Original_bgr, mask, alpha=0.45):
    color = np.zeros_like(Original_bgr); color[:] = (0, 255, 0)
    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    return np.where(mask3 > 0,
                    ((1 - alpha) * Original_bgr + alpha * color).astype(np.uint8),
                    Original_bgr)

def put_multiline(img, lines, org=(10, 25), scale=1.0, color=(0, 255, 255),
                  thickness=1, line_gap=30):
    x, y = org
    for line in lines:
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, color, thickness, cv2.LINE_AA)
        y += line_gap
    return img

# ---- 파라미터 저장/복원 ----
DEFAULT_PARAMS = {
    "R_MIN": 160, "G_MIN": 160, "B_MIN": 140,
    "BLUR_K_RAW": 0, "MORPH_K_RAW": 0,
    "LargestOnly": 1,
    "InvertMask": 1,
    "SideCutPercent": 25   # 🔥 좌우 잘라낼 비율 (0~50)
}
def load_params():
    try:
        with open(win_longpath(params_path), "r", encoding="utf-8") as f:
            d = json.load(f)
        return {**DEFAULT_PARAMS, **d}
    except Exception:
        return DEFAULT_PARAMS.copy()

def save_params(d):
    try:
        params_path.parent.mkdir(parents=True, exist_ok=True)
        tmp = params_path.with_suffix(".json.tmp")
        with open(win_longpath(tmp), "w", encoding="utf-8") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
        os.replace(win_longpath(tmp), win_longpath(params_path))
    except Exception as e:
        print(f"[WARN] 파라미터 저장 실패: {e}")

# ========= 비디오/스케일 =========
cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    raise FileNotFoundError(f"영상 열기 실패: {video_path}")
ret, first = cap.read()
if not ret:
    raise RuntimeError("첫 프레임 읽기 실패")
H, W = first.shape[:2]
mm_per_px = side_length_mm / W
area_mm2_per_px = mm_per_px ** 2
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
print(f"[INFO] 해상도 {W}x{H}, 총 프레임 {total_frames}, 1px = {mm_per_px:.5f} mm, 1px² = {area_mm2_per_px:.6f} mm²")

# ========= 트랙바 =========
cv2.namedWindow("Preview", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Preview", min(1500, W * 3), min(900, H * 2))

def _nothing(_): pass
cv2.createTrackbar("R_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("G_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("B_MIN",       "Preview", 0, 255, _nothing)
cv2.createTrackbar("BLUR_K",      "Preview", 0, 31, _nothing)
cv2.createTrackbar("MORPH_K",     "Preview", 0, 31, _nothing)
cv2.createTrackbar("LargestOnly", "Preview", 0, 1, _nothing)
cv2.createTrackbar("InvertMask", "Preview", 0, 1, _nothing)
cv2.createTrackbar("SideCutPercent", "Preview", 40, 50, _nothing)

# ---- 저장된 값으로 초기화 ----
p = load_params()
cv2.setTrackbarPos("R_MIN",       "Preview", int(p["R_MIN"]))
cv2.setTrackbarPos("G_MIN",       "Preview", int(p["G_MIN"]))
cv2.setTrackbarPos("B_MIN",       "Preview", int(p["B_MIN"]))
cv2.setTrackbarPos("BLUR_K",      "Preview", int(p["BLUR_K_RAW"]))
cv2.setTrackbarPos("MORPH_K",     "Preview", int(p["MORPH_K_RAW"]))
cv2.setTrackbarPos("LargestOnly", "Preview", int(p["LargestOnly"]))
cv2.setTrackbarPos("InvertMask", "Preview", int(p["InvertMask"]))
cv2.setTrackbarPos("SideCutPercent", "Preview", int(p["SideCutPercent"]))

# ========= 저장 루틴 =========
def save_all(R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, LARGEST):
    print("[INFO] 저장 시작...")
    base_folder.mkdir(parents=True, exist_ok=True)
    Original_folder.mkdir(parents=True, exist_ok=True)
    Binary_folder.mkdir(parents=True, exist_ok=True)

    cap2 = cv2.VideoCapture(str(video_path))
    if not cap2.isOpened():
        print("[ERR] 저장용 VideoCapture 열기 실패")
        return

    frame_list, area_px_list, area_mm2_list = [], [], []
    j = 0
    write_fail = 0
    saved_count = 0

    every_n = max(1, int(SAVE_EVERY))
    offset_n = int(SAVE_OFFSET) % every_n

    while True:
        ret2, fr2 = cap2.read()
        if not ret2:
            break

        # ✅ 절대 임계값 방식 사용
        m2 = preprocess_and_mask(fr2, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, LARGEST)

        if LARGEST:
            num_labels, _, stats, _ = cv2.connectedComponentsWithStats(m2, connectivity=8)
            px = int(stats[1:, cv2.CC_STAT_AREA].max()) if num_labels > 1 else 0
        else:
            px = int(np.count_nonzero(m2))

        is_target = ((j - offset_n) % every_n == 0)
        if EXCLUDE_FRAME0 and j == 0:
            is_target = False

        if not CSV_ONLY_SAVED or is_target:
            frame_list.append(j)
            area_px_list.append(px)
            area_mm2_list.append(px * area_mm2_per_px)

        if is_target:
            f_name = f"Original_{j:05d}.png"
            m_name = f"Binary_{j:05d}.png"
            ok1 = safe_write_image(Original_folder / f_name, fr2)
            ok2 = safe_write_image(Binary_folder / m_name, m2)
            if not ok1 or not ok2:
                write_fail += 1
                if not ok1: print(f"[WARN] 원본 저장 실패: {Original_folder / f_name}")
                if not ok2: print(f"[WARN] 마스크 저장 실패: {Binary_folder / m_name}")
            else:
                saved_count += 1
                if DEBUG_PRINT_SAVED:
                    print(f"[SAVE] frame {j} saved (every={every_n}, offset={offset_n})")

        if j % 100 == 0:
            print(f"  processed frame {j}")
        j += 1

    cap2.release()

    try:
        df = pd.DataFrame({
            "frame": frame_list,
            "area_px": area_px_list,
            "area_mm2": area_mm2_list
        })
        csv_tmp = base_folder / f"{video_path.stem}_area_tmp.csv"
        df.to_csv(win_longpath(csv_tmp), index=False, encoding="utf-8-sig")
        os.replace(win_longpath(csv_tmp), win_longpath(csv_path))
    except Exception as e:
        print(f"[ERR] CSV 저장 실패: {e}")

    print(f"[INFO] 저장 완료:\n"
          f" - Original: {Original_folder} (saved {saved_count} files)\n"
          f" - Binary  : {Binary_folder} (saved {saved_count} files)\n"
          f" - CSV rows: {len(frame_list)} "
          f"({'only saved frames' if CSV_ONLY_SAVED else 'all frames'})\n"
          f" - Interval: every {every_n} frames (offset {offset_n})\n"
          f" - write_fail: {write_fail}")

# ========= 프리뷰 루프 =========
idx = 0; paused = False

def read_slider_raw():
    return {
        "R_MIN": cv2.getTrackbarPos("R_MIN", "Preview"),
        "G_MIN": cv2.getTrackbarPos("G_MIN", "Preview"),
        "B_MIN": cv2.getTrackbarPos("B_MIN", "Preview"),
        "BLUR_K_RAW": cv2.getTrackbarPos("BLUR_K", "Preview"),
        "MORPH_K_RAW": cv2.getTrackbarPos("MORPH_K", "Preview"),
        "LargestOnly": cv2.getTrackbarPos("LargestOnly", "Preview"),
        "InvertMask": cv2.getTrackbarPos("InvertMask", "Preview"),
        "SideCutPercent": cv2.getTrackbarPos("SideCutPercent", "Preview"),  # 🔥 추가
    }

try:
    while True:
        idx = idx % max(1, total_frames)
        cap = cv2.VideoCapture(str(video_path))
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read(); cap.release()
        if not ret: idx += 1; continue

        # 슬라이더 값 읽기 (원본 & 변환값)
        raw = read_slider_raw()
        R_MIN   = raw["R_MIN"]
        G_MIN   = raw["G_MIN"]
        B_MIN   = raw["B_MIN"]
        BLUR_K  = odd_from_slider(raw["BLUR_K_RAW"])
        MORPH_K = odd_from_slider(raw["MORPH_K_RAW"])
        use_largest = bool(raw["LargestOnly"])

        mask = preprocess_and_mask(frame, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, use_largest)

        # 반전
        if raw["InvertMask"]:
            mask = cv2.bitwise_not(mask)
        
        # 좌우 컷
        cut_percent = raw["SideCutPercent"] / 100.0
        H, W = mask.shape
        left_cut = int(W * cut_percent)
        right_cut = int(W * (1 - cut_percent))
        region = np.zeros_like(mask)
        region[:, left_cut:right_cut] = 255
        mask = cv2.bitwise_and(mask, region)
        overlay = overlay_mask(frame, mask, alpha=0.45)
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        panel = np.hstack([frame, overlay, mask_bgr])

        lines = [
            f"Frame {idx}/{total_frames-1}",
            "Controls: SPACE=Pause/Play, S=Save(Original+Binary+CSV), R=Restart, Q=Quit",
            f"Save interval: every {SAVE_EVERY} frames (offset {SAVE_OFFSET}), CSV={'saved only' if CSV_ONLY_SAVED else 'all frames'}",
            "Tip: The Preview window must be active for keystrokes to be recognised.",
            f"Params: R_MIN={R_MIN}  G_MIN={G_MIN}  B_MIN={B_MIN}  BLUR={BLUR_K}  MORPH={MORPH_K}  LargestOnly={int(use_largest)}",
        ]

        panel = put_multiline(panel, lines, org=(10, 30))
        cv2.imshow("Preview", panel)
        
        key = cv2.waitKey(1 if not paused else 30) & 0xFF
        if key == ord('q'):
            # 종료 직전, 마지막 값 저장
            save_params(raw)
            break
        elif key == 32:  # SPACE
            paused = not paused
        elif key == ord('s'):
            print("[INFO] 's' 감지 → 저장을 시작합니다. (Preview 창 포커스 필요)")
            save_all(R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, use_largest)
            # 저장 시점에도 현재 민감도 저장
            save_params(raw)
        elif key == ord('r'):  # (NEW) R 키 → 영상 재시작
            print("[INFO] 'r' 감지 → 영상 처음부터 재시작")
            idx = 0
            paused = False

        if not paused:
            idx += 1
        else:
            time.sleep(0.02)
finally:
    cv2.destroyAllWindows()
    print("[INFO] 종료")


[INFO] 해상도 720x650, 총 프레임 242, 1px = 0.09167 mm, 1px² = 0.008403 mm²
[INFO] 종료
